In [2]:
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt


mat_data = sio.loadmat('ex8/ex8_movies.mat')

# Y is a 1682x943 matrix, containing ratings (1-5) of 1682 movies on 943 users
Y = mat_data['Y']

# R is a 1682x943 matrix, where R(i,j) = 1 if and only if user j gave a rating to movie i
R = mat_data['R']

print('Average rating for movie 1 (Toy Story): {} / 5'.format(np.mean(Y[0, np.nonzero(R[0, ])])))

# We can "visualize" the ratings matrix by plotting it with imshow
plt.figure()
plt.imshow(Y, aspect='auto')
plt.ylabel('Movies')
plt.xlabel('Users')
plt.show()

Average rating for movie 1 (Toy Story): 3.8783185840707963 / 5


<Figure size 640x480 with 1 Axes>

In [5]:
def cofi_cost_func(params, Y, R, num_users, num_movies, num_features, l):
    """
    Returns the cost and gradient for the collaborative filtering problem.
    Parameters
    ----------
    params : ndarray, shape ((num_users + num_movies) * num_features,)
        Parameters for collaborative filtering.
    Y : ndarray, shape (num_movies, num_users)
        Ratings, where n_movies is the number of movies and n_users is the number of users.
    R : ndarray, shape (num_movies, num_users)
        R(i,j) = 1 if and only if user j gave a rating to movie i.
    num_users : int
        Number of users.
    num_movies : int
        Number of movies.
    num_features : int
        Number of features.
    l : float, shape ((num_users + num_movies) * num_features,)
        Regularization parameter.
    Returns
    -------
    J : numpy.float64
        The cost for collaborative filtering.
    grad : ndarray, shape ((num_users + num_movies) * num_features,)
        The gradient for collaborative filtering.
    """
    X = params[0:num_movies*num_features].reshape((num_movies, num_features))
    Theta = params[num_movies*num_features: ].reshape((num_users, num_features))

    J = 0.5 * np.sum(np.sum(R * np.square(X.dot(Theta.T) - Y)))
    X_grad = (R * (X.dot(Theta.T) - Y)).dot(Theta)
    Theta_grad = (R * (X.dot(Theta.T) - Y)).T.dot(X)

    J = J + 0.5 * l * np.sum(np.square(Theta)) + 0.5 * l * np.sum(np.square(X))
    X_grad = X_grad + l * X
    Theta_grad = Theta_grad + l * Theta
    print(X_grad)
    print(X_grad.ravel())

    grad = np.hstack((X_grad.ravel(), Theta_grad.ravel()))

    return J, grad

In [6]:
mat_data = sio.loadmat('ex8/ex8_movieParams.mat')
X = mat_data['X']
Theta = mat_data['Theta']
num_users = mat_data['num_users'].ravel()[0]
num_movies = mat_data['num_movies'].ravel()[0]
num_features = mat_data['num_features'].ravel()[0]

# Reduce the data set size so that this runs faster
num_users = 4
num_movies = 5
num_features = 3
X = X[0:num_movies, 0:num_features]
Theta = Theta[0:num_users, 0:num_features]
Y = Y[0:num_movies, 0:num_users]
R = R[0:num_movies, 0:num_users]

J, grad = cofi_cost_func(np.hstack((X.flatten(), Theta.flatten())), Y, R, num_users, num_movies, num_features, 0)
print('Cost at loaded parameters:', J)
print('(this value should be about 22.22)')

[[-2.52899165  7.57570308 -1.89979026]
 [-0.56819597  3.35265031 -0.52339845]
 [-0.83240713  4.91163297 -0.76677878]
 [-0.38358278  2.26333698 -0.35334048]
 [-0.80378006  4.74271842 -0.74040871]]
[-2.52899165  7.57570308 -1.89979026 -0.56819597  3.35265031 -0.52339845
 -0.83240713  4.91163297 -0.76677878 -0.38358278  2.26333698 -0.35334048
 -0.80378006  4.74271842 -0.74040871]
Cost at loaded parameters: 22.224603725685675
(this value should be about 22.22)


In [8]:

J, grad = cofi_cost_func(np.hstack((X.flatten(), Theta.flatten())), Y, R, num_users, num_movies, num_features, 1.5)
print('Cost at loaded parameters (lambda = 1.5):', J)
print('(this value should be about 31.34)')

[[-0.95596339  6.97535514 -0.10861109]
 [ 0.60308088  2.77421145  0.25839822]
 [ 0.12985616  4.0898522  -0.89247334]
 [ 0.29684395  1.06300933  0.66738144]
 [ 0.60252677  4.90185327 -0.19747928]]
[-0.95596339  6.97535514 -0.10861109  0.60308088  2.77421145  0.25839822
  0.12985616  4.0898522  -0.89247334  0.29684395  1.06300933  0.66738144
  0.60252677  4.90185327 -0.19747928]
Cost at loaded parameters (lambda = 1.5): 31.34405624427422
(this value should be about 31.34)


In [20]:
def load_movie_list():
    """
    Load movie list from movie_ids.txt.
    Returns
    -------
    movie_list : list
        The movie list.
    """
    movie_list =[]
    with open("ex8/movie_ids.txt",encoding = "ISO-8859-1") as f:
        for line in f:
            movie_list.append(line[line.index(' ') + 1:].rstrip())
    return movie_list

In [21]:
movie_list = load_movie_list()

# Initialize my ratings
my_ratings = np.zeros(len(movie_list), dtype=np.int)
my_ratings[0] = 4
my_ratings[97] = 2
my_ratings[6] = 3
my_ratings[11] = 5
my_ratings[53] = 4
my_ratings[63] = 5
my_ratings[65] = 3
my_ratings[68] = 5
my_ratings[182] = 4
my_ratings[225] = 5
my_ratings[354] = 5

print('New user ratings:')
for i in np.argwhere(my_ratings > 0).ravel():
    print('Rated {} for {}'.format(my_ratings[i], movie_list[i]))

New user ratings:
Rated 4 for Toy Story (1995)
Rated 3 for Twelve Monkeys (1995)
Rated 5 for Usual Suspects, The (1995)
Rated 4 for Outbreak (1995)
Rated 5 for Shawshank Redemption, The (1994)
Rated 3 for While You Were Sleeping (1995)
Rated 5 for Forrest Gump (1994)
Rated 2 for Silence of the Lambs, The (1991)
Rated 4 for Alien (1979)
Rated 5 for Die Hard 2 (1990)
Rated 5 for Sphere (1998)


In [22]:
def normalize_ratings(Y, R):
    """
    Preprocess data by subtracting mean rating for every movie (every row).

    Parameters
    ----------
    Y : ndarray, shape (n_movies, n_users)
        Ratings, where n_movies is the number of movies and n_users is the number of users.
    R : ndarray, shape (n_samples, n_users)
        R(i,j) = 1 if and only if user j gave a rating to movie i.

    Returns
    -------
    Y_norm : ndarray, shape (n_movies, n_users)
        Normalized Y with each movie has am average rating of 0.
    Y_mean : ndarray, shape (n_movies, 1)
        Mean rating for each movie.
    """
    m = Y.shape[0]
    Y_mean = np.zeros(m)
    Y_norm = np.zeros(Y.shape)

    for i in range(m):
        idx = np.nonzero(R[i, ] == 1)
        Y_mean[i] = np.mean(Y[i, idx])
        Y_norm[i, idx] = Y[i, idx] - Y_mean[i]

    return Y_norm, Y_mean

In [24]:
import scipy.optimize as opt


mat_data = sio.loadmat('ex8/ex8_movies.mat')

# Y is a 1682x943 matrix, containing ratings (1-5) of 1682 movies on 943 users
Y = mat_data['Y']
# R is a 1682x943 matrix, where R(i,j) = 1 if and only if user j gave a rating to movie i
R = mat_data['R']

# Add our own ratings to the data matrix
Y = np.hstack((my_ratings.reshape(len(movie_list), 1), Y))
R = np.hstack((my_ratings.reshape(len(movie_list), 1) != 0, R))

# Normalize Ratings
Y_norm, Y_mean = normalize_ratings(Y, R)

# Useful Values
num_users = Y.shape[1]
num_movies = Y.shape[0]
num_features = 10

# Set Initial Parameters (Theta, X)
X = np.random.randn(num_movies, num_features)
Theta = np.random.randn(num_users, num_features)

initial_parameters = np.hstack((X.flatten(), Theta.flatten()))

# Set Regularization
l = 10
result = opt.minimize(fun=cofi_cost_func, x0=initial_parameters,
                      args=(Y_norm, R, num_users, num_movies, num_features, l),
                      method='CG', jac=True, options={'maxiter': 150})

X = result.x[0:num_movies * num_features].reshape((num_movies, num_features))
Theta = result.x[num_movies * num_features:].reshape((num_users, num_features))

print('Recommender system learning completed.')

[[-154.62380686  -83.76198711 -328.09939518 ...  186.70004349
   145.97474833  -47.04288477]
 [  32.0119899   138.26258171   98.49014726 ...  -47.91057552
   -86.63028104 -107.14753016]
 [ 242.57419796  193.83498522   -9.25996859 ...   21.34750489
   260.06621809   35.09427771]
 ...
 [  -8.01410831    0.87769326   12.66905633 ...    8.68818482
   -16.36644619   11.07867726]
 [  14.96253877  -16.8223494    11.23851312 ...   -0.74169593
    10.97693525    7.35845365]
 [   9.60280422    1.04561362   -2.65473712 ...   40.27744452
     0.6694533   -12.34372092]]
[-154.62380686  -83.76198711 -328.09939518 ...   40.27744452    0.6694533
  -12.34372092]
[[-154.62380686  -83.76198711 -328.09939518 ...  186.70004349
   145.97474833  -47.04288477]
 [  32.0119899   138.26258171   98.49014726 ...  -47.91057552
   -86.63028104 -107.14753016]
 [ 242.57419796  193.83498522   -9.25996859 ...   21.34750489
   260.06621809   35.09427771]
 ...
 [  -8.01410831    0.87769326   12.66905633 ...    8.68818482


[[-0.61858217 -1.60942001  0.47455282 ... -0.49712699  8.26270172
  -4.23006408]
 [ 0.80475442  1.4458042   0.06819779 ...  0.81784893  1.94181624
  -0.97559565]
 [-0.41455117  3.03995414 -1.64708938 ...  0.93260097 -1.80309401
   0.4183722 ]
 ...
 [-0.36618395  1.22112676  2.08583936 ...  1.37745854 -3.05983147
   1.60044579]
 [ 1.64173077 -2.31644778  1.78718006 ...  0.42779703  2.15559932
   1.67696857]
 [ 1.75947411  0.19963811 -0.48329288 ...  7.5142541   0.12534665
  -2.31974812]]
[-0.61858217 -1.60942001  0.47455282 ...  7.5142541   0.12534665
 -2.31974812]
[[  4.46052706  -0.92452099   9.10979362 ...  12.72662152 -10.03297058
    9.44104581]
 [  3.85661931  -2.03356957   7.12416092 ...   1.06081173  -7.92651494
    9.01178246]
 [ -1.3160402    4.05196208  -0.97876109 ...  -0.11821116  -0.5075748
   -1.30258945]
 ...
 [ -0.21242706   0.66933389   1.15192732 ...   0.78284801  -1.6868791
    0.91151861]
 [  0.89706116  -1.31918688   1.02075312 ...   0.27068398   1.18926317
    0.9

[[-0.61825817  1.4225512  -2.40567275 ... -1.39533291 -1.72007234
   0.69653406]
 [ 0.14273115  1.0585729   0.48096478 ...  0.48392845  1.52601059
  -1.31324303]
 [ 0.17709178 -0.23593439  0.35417351 ... -0.68263034  0.9035809
  -1.14926095]
 ...
 [ 0.01081655 -0.02698475 -0.04734124 ... -0.03386994  0.07074596
  -0.04187233]
 [-0.03251355  0.06217763 -0.04703875 ... -0.01294652 -0.04670948
  -0.04415438]
 [-0.04179756 -0.00583943  0.01160412 ... -0.17346668 -0.0026891
   0.05429848]]
[-0.61825817  1.4225512  -2.40567275 ... -0.17346668 -0.0026891
  0.05429848]
[[-0.43708378 -0.30561295 -0.57008229 ... -1.46582829  0.82659307
  -1.33552849]
 [ 0.62562412 -0.29225632  1.3458545  ...  0.07908454 -0.19951853
   0.77089961]
 [-0.1541765  -0.27444126 -0.05424855 ... -0.42269425  0.74498594
  -0.7234857 ]
 ...
 [ 0.016769   -0.04033487 -0.07127751 ... -0.05098459  0.10722323
  -0.06392095]
 [-0.05355043  0.08479497 -0.06501165 ... -0.01525611 -0.07350888
  -0.06129914]
 [-0.0648388  -0.00732

[[ 5.10689714e-01 -9.07531075e-01  1.62552598e-01 ...  1.09778503e+00
  -9.09971085e-01  1.22130583e+00]
 [ 4.66108296e-02  8.08410780e-02 -1.51867342e-01 ... -8.83625127e-03
   2.55727342e-01 -2.89495998e-01]
 [ 6.14122805e-02 -5.99624856e-02  5.71225532e-02 ... -2.44241407e-02
  -2.84463669e-01  2.41403178e-01]
 ...
 [ 4.39924693e-03 -2.57296270e-03 -1.01941079e-02 ... -5.05333226e-03
   1.96202361e-02 -1.24949472e-02]
 [-4.14335563e-03  1.38516798e-02 -1.07729988e-02 ... -3.23252745e-03
  -8.38605515e-03 -1.00937425e-02]
 [-7.48635429e-03 -1.50714773e-03  1.91231169e-03 ... -3.23800520e-02
  -6.14116366e-04  1.01658189e-02]]
[ 5.10689714e-01 -9.07531075e-01  1.62552598e-01 ... -3.23800520e-02
 -6.14116366e-04  1.01658189e-02]
[[-6.68950288e-01  1.12833063e+00 -4.82501629e-02 ... -1.30657920e+00
   1.55887103e+00 -1.79575931e+00]
 [ 9.36083465e-02 -3.54624477e-02  4.15741924e-02 ...  5.86477074e-02
   9.72297697e-02 -2.99880079e-02]
 [ 5.35266044e-02  2.10002886e-02 -3.33925838e-02 .

[[-8.45717172e-03  2.21276680e-01 -9.37385024e-02 ... -3.06278298e-01
   4.67392440e-01 -4.59425760e-01]
 [-4.82581774e-02  3.88456145e-02 -2.51273699e-02 ... -1.52726058e-02
   8.85456568e-02 -1.32399489e-01]
 [ 7.50134052e-02 -1.29966481e-01 -3.27916941e-02 ... -1.06094668e-01
  -2.56062562e-01  3.75806323e-02]
 ...
 [ 2.25072474e-03 -3.41445683e-03 -7.42457445e-03 ... -4.71947355e-03
   1.23583775e-02 -7.59515695e-03]
 [-5.28753439e-03  8.24174856e-03 -6.38499162e-03 ... -1.42985179e-03
  -7.33611777e-03 -5.98672982e-03]
 [-6.28172848e-03 -8.50237082e-04  1.85195723e-03 ... -2.56807789e-02
  -3.86731665e-04  7.97304766e-03]]
[-0.00845717  0.22127668 -0.0937385  ... -0.02568078 -0.00038673
  0.00797305]
[[ 5.38976549e-02 -3.34933898e-01  8.16764622e-02 ...  6.98429302e-01
  -2.88851953e-01  3.83628917e-01]
 [-9.00004156e-02  9.24500902e-03  2.37398793e-02 ... -5.86300047e-02
   6.43032011e-02  1.29926748e-02]
 [ 3.37750762e-02 -9.20751978e-02 -5.49372970e-02 ... -1.03982897e-01
  -2.

...

[[-3.10498263e-02  8.82436370e-02 -9.25036322e-02 ...  2.66110867e-02
  -7.16130393e-02 -8.11748079e-02]
 [-3.74878735e-02  1.82999970e-02 -3.80229505e-02 ... -4.71369128e-03
   3.53500178e-02 -2.53133911e-02]
 [-1.89601658e-03  3.05889776e-02  4.50764435e-03 ... -4.45649492e-03
  -3.53973948e-03 -2.91245521e-02]
 ...
 [-1.65990823e-04 -9.58043580e-04 -1.00697214e-03 ... -9.59979575e-04
  -1.38953448e-04  2.13163090e-04]
 [-1.81099056e-03  4.55956111e-04 -4.28806320e-04 ...  2.21430990e-04
  -1.57386874e-03 -3.71589493e-04]
 [-1.32769099e-03 -3.38442802e-05  4.11982480e-04 ... -4.81415253e-03
   4.36271136e-05  1.45410767e-03]]
[-3.10498263e-02  8.82436370e-02 -9.25036322e-02 ... -4.81415253e-03
  4.36271136e-05  1.45410767e-03]
[[-2.04087968e-02 -5.67387235e-04 -8.38574891e-02 ...  4.94575760e-03
  -1.30750573e-01 -8.10797814e-03]
 [-2.35027299e-02 -1.61897096e-02  3.53971522e-03 ...  2.39615615e-02
  -3.94526731e-02  3.20181437e-02]
 [ 1.08125056e-02  3.45726708e-02 -8.91440879e-04 .

[[-2.30591370e-02  1.50189148e-02 -1.28270318e-02 ... -8.34681611e-03
   1.53857477e-02  4.18460163e-03]
 [ 1.59108756e-04  5.59893316e-03 -6.52275923e-03 ... -1.69729803e-02
   1.71673556e-02 -9.59923017e-03]
 [ 1.25650801e-02  8.45944158e-03  1.25941970e-02 ... -9.43341832e-03
  -1.05142306e-02 -8.67369036e-03]
 ...
 [-1.64071936e-04 -4.56065205e-04 -4.64938600e-04 ... -4.07892018e-04
  -3.14484632e-04  3.00530796e-04]
 [-8.89205281e-04  1.64864547e-04 -1.57810033e-04 ...  1.36889283e-04
  -7.88881282e-04 -1.48609284e-04]
 [-6.94533441e-04  1.10038151e-05  2.01979572e-04 ... -2.33957832e-03
   6.62822536e-05  7.01385630e-04]]
[-2.30591370e-02  1.50189148e-02 -1.28270318e-02 ... -2.33957832e-03
  6.62822536e-05  7.01385630e-04]
[[-5.15137427e-02  5.47810241e-02 -6.24012596e-02 ... -6.57218311e-02
   1.23436880e-02  6.96419710e-03]
 [ 6.08561475e-03  1.75418713e-02 -1.73013792e-02 ... -1.96817912e-02
   4.21597913e-02 -2.48930044e-02]
 [ 4.67054427e-03  1.18432380e-02  1.04720224e-02 .

[[-1.52514973e-02  1.30786560e-02 -1.73669102e-02 ... -1.11361526e-03
  -1.82135035e-02  1.30352236e-02]
 [-3.34252943e-04  5.84261773e-03 -8.68450384e-03 ...  1.43405311e-03
   5.43635187e-04  3.73087793e-03]
 [-5.32645028e-04  5.57234615e-03  3.22866539e-03 ...  6.94321150e-03
   7.19590629e-03 -1.26206834e-02]
 ...
 [ 2.01234629e-06  2.74070137e-04  2.97646757e-04 ...  3.00344361e-04
  -9.42719750e-05  4.42003495e-05]
 [ 5.02107992e-04 -1.81794773e-04  1.67939227e-04 ... -3.51127597e-05
   4.22413232e-04  1.34929294e-04]
 [ 3.57860406e-04  1.86359234e-05 -1.15652559e-04 ...  1.35691149e-03
   2.64840548e-06 -4.11638163e-04]]
[-1.52514973e-02  1.30786560e-02 -1.73669102e-02 ...  1.35691149e-03
  2.64840548e-06 -4.11638163e-04]
[[ 2.83732107e-03 -5.03796749e-03 -2.13108518e-02 ... -8.13469670e-03
  -5.20859075e-03 -1.92171765e-02]
 [ 4.21813678e-03  5.97274395e-03 -1.27302666e-02 ...  6.66415762e-03
  -2.38788339e-03  6.45633601e-03]
 [-1.71040302e-03  7.25889169e-03  2.71941662e-03 .

[[-1.24206869e-02  6.24332263e-03  5.15833164e-03 ...  5.42415609e-02
  -3.29929117e-02  4.57788974e-02]
 [-8.43167585e-03  6.29116750e-03 -7.25478456e-03 ... -6.77786124e-03
   8.17285832e-03 -1.37294089e-02]
 [-9.97748806e-03  2.09821504e-03  6.14827017e-03 ... -2.14125968e-03
   1.09140348e-02 -8.83192194e-03]
 ...
 [ 2.24857435e-05 -3.23345740e-04 -3.56365666e-04 ... -3.68986852e-04
   1.84063051e-04 -1.10929728e-04]
 [-5.87099024e-04  2.18752350e-04 -2.01866679e-04 ...  3.81211898e-05
  -4.92317337e-04 -1.61325668e-04]
 [-4.00006733e-04 -3.26209407e-05  1.34822626e-04 ... -1.59028768e-03
  -2.07822937e-05  4.84577628e-04]]
[-1.24206869e-02  6.24332263e-03  5.15833164e-03 ... -1.59028768e-03
 -2.07822937e-05  4.84577628e-04]
[[-1.29317896e-02  1.00370689e-02 -3.39857657e-03 ...  2.80028060e-02
  -1.99372543e-02  3.12997055e-02]
 [-3.16828838e-03  5.17920077e-03 -5.69555037e-03 ... -5.13611600e-03
   7.34681495e-03 -8.15532631e-03]
 [-6.27036356e-03  2.64054597e-03  4.06890417e-03 .

[[ 6.07266504e-03 -5.53086865e-03 -3.59293661e-03 ... -1.67934768e-02
   9.83744470e-04  3.80442586e-03]
 [ 2.16900858e-03  9.21876575e-04 -6.56615933e-04 ... -1.88834832e-03
   2.19554584e-03  3.59866503e-03]
 [ 3.60022446e-03 -2.13501579e-03  4.35642836e-04 ...  8.47690602e-03
   2.62468462e-03  6.26496688e-04]
 ...
 [-4.52265671e-05  6.49593875e-04  7.15957994e-04 ...  7.41306760e-04
  -3.69969884e-04  2.22984535e-04]
 [ 1.18046928e-03 -4.32590248e-04  3.99400443e-04 ... -8.01303556e-05
   9.91721091e-04  3.20182194e-04]
 [ 8.00156529e-04  6.74903457e-05 -2.70742968e-04 ...  3.19516349e-03
   4.49587558e-05 -9.73978211e-04]]
[ 6.07266504e-03 -5.53086865e-03 -3.59293661e-03 ...  3.19516349e-03
  4.49587558e-05 -9.73978211e-04]
[[-1.74884815e-02  1.15575237e-02  3.71914269e-03 ...  2.14745808e-02
  -2.52996935e-02 -2.70344116e-03]
 [-3.64856705e-03  5.28444653e-03 -7.66599271e-03 ... -9.99415092e-03
  -8.86127823e-03 -2.52977569e-02]
 [-1.08570419e-02  8.16625058e-03  8.35524827e-03 .

In [27]:
p = X.dot(Theta.T)
my_predictions = p[:, 0] + Y_mean
idx = np.argsort(my_predictions)[::-1]
print('Top recommendations for you:')
for i in range(10):
    print('Predicting rating {0:.1f} for movie {1:s}'.format(my_predictions[idx[i]], movie_list[idx[i]]))

print('\nOriginal ratings provided:')
for i in np.argwhere(my_ratings > 0).ravel():
    print('Rated {} for {}'.format(my_ratings[i], movie_list[i]))


Top recommendations for you:
Predicting rating 5.0 for movie Aiqing wansui (1994)
Predicting rating 5.0 for movie Santa with Muscles (1996)
Predicting rating 5.0 for movie Star Kid (1997)
Predicting rating 5.0 for movie Someone Else's America (1995)
Predicting rating 5.0 for movie Marlene Dietrich: Shadow and Light (1996)
Predicting rating 5.0 for movie They Made Me a Criminal (1939)
Predicting rating 5.0 for movie Prefontaine (1997)
Predicting rating 5.0 for movie Entertaining Angels: The Dorothy Day Story (1996)
Predicting rating 5.0 for movie Great Day in Harlem, A (1994)
Predicting rating 5.0 for movie Saint of Fort Washington, The (1993)

Original ratings provided:
Rated 4 for Toy Story (1995)
Rated 3 for Twelve Monkeys (1995)
Rated 5 for Usual Suspects, The (1995)
Rated 4 for Outbreak (1995)
Rated 5 for Shawshank Redemption, The (1994)
Rated 3 for While You Were Sleeping (1995)
Rated 5 for Forrest Gump (1994)
Rated 2 for Silence of the Lambs, The (1991)
Rated 4 for Alien (1979)
R